In [125]:
# объявим где хранятся исходные данные
PATH_TRAIN = 'input/train.csv'
PATH_TEST = 'input/test.csv'

In [126]:
# объявим куда сохраним результат
PATH_PRED = 'pred.csv'

In [127]:
import codecs

In [128]:
## Из тренировочного набора собираем статистику о встречаемости слов

# создаем словарь для хранения статистики
word_stat_dict = {}

In [129]:
# открываем файл на чтение в режиме текста
fl = codecs.open(PATH_TRAIN, 'r', encoding='utf8')

# считываем первую строчку - заголовок (она нам не нужна)
fl.readline()

'Id,Sample,Prediction\n'

In [130]:
# в цикле читаем строчки из файла
i=0
all_words = {}
for line in fl:
    # разбиваем строчку на три строковые переменные
    Id, Sample, Prediction = line.strip().split(',')
    # строковая переменная Prediction - содержит в себе словосочетание из 2 слов, разделим их

    #word01, word02 = Sample.split(' ')
    word1, word2 = Prediction.split(' ')
    
    
    # all_words словарь всех слов с их частотами
    if word2 not in all_words:
        all_words[word2] = 0
    all_words[word2] += 1
    
    # word_stat_dict - словарь, где ключ это первое слово, а значение это словарь вторых слов, где value это частоты второго слова после первого
    if word1 not in word_stat_dict:
        word_stat_dict[word1] = {}
    # если текущее слово еще не встречалось, то добавим его в словарь и установим счетчик этого слова в 0
    if word2 not in word_stat_dict[word1]:
        word_stat_dict[word1][word2] = 0
    # увеличим значение счетчика по текущему слову на 1
    word_stat_dict[word1][word2] += 1


In [131]:
# закрываем файл
fl.close()

In [134]:
len(word_stat_dict)
len(all_words)

82346

In [135]:
a=[['a','b','c'],[1,2,3]]
sorted_all_words = dict()
for j in sorted(all_words):
    sorted_all_words[j] = all_words[j]

In [136]:
count = 0
for i in sorted_all_words.keys():
    if sorted_all_words[i]>count:
        count = sorted_all_words[i]
        frequent_word = i
frequent_word, count

('его', 8899)

In [137]:
## Выполняем предсказание

# открываем файл на чтение в режиме текста
fl = codecs.open(PATH_TEST, 'r', encoding='utf8')

# считываем первую строчку - заголовок (она нам не нужна)
fl.readline()
# открываем файл на запись в режиме текста
out_fl = codecs.open(PATH_PRED, 'w',encoding='utf8')

# записываем заголовок таблицы
out_fl.write('Id,Prediction\n')
# в цикле читаем строчки из тестового файла
for line in fl:
    # разбиваем строчку на две строковые переменные
    Id, Sample = line.strip().split(',')
    # строковая переменная Sample содержит в себе полностью первое слово и кусок второго слова, разделим их
    word1, word2_chunk = Sample.split(' ')
   
    
    word_pred=''
    count = 0
    
    # flag = 1 нашли в word_stat_dict
    flag = 1
    # совпадение первых слов
    if word1 in word_stat_dict:
        #поиск совпадений начал вторых слов
        for i in word_stat_dict[word1].keys():
            if word2_chunk == i[:len(word2_chunk)]:
                if word_stat_dict[word1][i]>count:
                    count = word_stat_dict[word1][i]
                    word_pred = i
            else:
                if count>0:
                    break
                    
    # если в word_stat_dict не нашли
    if count == 0:
        flag = 0
        for i in sorted_all_words.keys():
            if word2_chunk == i[:len(word2_chunk)]:
                if sorted_all_words[i]>count:
                    count = sorted_all_words[i]
                    word_pred = i
            else:
                if count>0:
                    break


    # если не нашли, пишем наиболее часто встречающееся слово
    if count == 0:
        word_pred = frequent_word

    # записываем в word_stat_dict новое словосочетание
    if flag == 0:
        if word1 not in word_stat_dict:
            word_stat_dict[word1] = {}
        word_stat_dict[word1][word_pred] = 1

    
    #запись ответа в файл
    out_fl.write('%s,%s %s\n' % (Id, word1, word_pred))
        
# закрываем файлы
fl.close()
out_fl.close()